## 정수 인코딩(Integer Encoding

### dictionary 사용하기

In [1]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

여러 문장이 함께 있는 텍스트 데이터로부터 문장 토큰화 수행

In [3]:
# 문장 토큰화
text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


텍스트 데이터가 문장 단위로 토큰화됨.   
이제 정제 작업을 병행하며 단어 토큰화 수행

In [4]:
# 정제와 단어 토큰화
vocab = {} # 파이썬의 dictionary 자료형
sentences = []
stop_words = set(stopwords.words('english'))

for i in text:
    sentence = word_tokenize(i) # 단어 토큰화를 수행합니다.
    result = []

    for word in sentence: 
        word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄입니다.
        if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거합니다.
            if len(word) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거합니다.
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0 
                vocab[word] += 1
    sentences.append(result) 
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


텍스트를 숫자로 바꾸는 단계라는 것은 본격적으로 자연어 처리 작업에 들어간다는 의미.  
따라서 단어가 텍스트일 때만 할 수 있는 최대한의 전처리를 끝내야함.   
- 동일한 단어가 대문자로 표기되었다는 이유로 서로 다른 단어로 카운트 되는 일이 없도록 모든 단어를 소문자로 변환.
- NLP에서 크게 의미를 갖지 못하는 불용어와 길이가 짧은 단어를 제거하는 방법사용

현재 vocab에는 중복을 제거한 단어와 각 단어에 대한 빈도수가 기록되어 잇음. vocab 출력

In [5]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


단어를 key로, 빈도를 value로 저장. vocab에 단어를 입력하면 빈도수 출력

In [6]:
print(vocab["barber"]) # 'barber'라는 단어의 빈도수 출력

8


빈도수가 높은 대로 출력

In [7]:
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


이제 높은 빈도수 가진 단어일수록 낮은 정수 인덱스 부여.

In [8]:
word_to_index = {}
i=0
for (word, frequency) in vocab_sorted :
    if frequency > 1 : # 정제(Cleaning) 챕터에서 언급했듯이 빈도수가 적은 단어는 제외한다.
        i=i+1
        word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


index=1 인 것이 가장 빈도수가 높은 단어.   
이런 작업을 수행하는 동시에 빈도수가 적은 단어를 제외시키는 작업 처리 -> 빈도가 낮은 단어는 NLP에서 의미를 가지지 않을 가능성이 높기 때문

NLP를 하다보면 텍스트 데이터에 있는 모든 단어를 모두 사용하기보다는 빈도수가 높은 n개의 단어만 사용하고 싶은 경우가 많음. 따라서 빈도수 상위 n개 단어만 사용하고 싶으면 vocab에서 정수값이 1부터 n까지인 단어들만 사용하면 됨.   
여기서는 상위 5개 단어만 사용한다 가정.

In [9]:
vocab_size = 5
words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거
for w in words_frequency:
    del word_to_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


word_to_index에는 빈도 수 높은 상위 5개 단어만 저장되어 있음. 이제 word_to_index를 이용해 단어 토큰화가 된 상태로 저장된 sentences에 있는 각 단어를 정수로 바꾸는 작업을 함.   

예를 들어 sentences에서 첫번째 문장은 ['barber', 'person']이었는데, 이 문장에 대해서는 [1, 5]로 인코딩합니다. 그런데 두번째 문장인 ['barber', 'good', 'person']에는 더 이상 word_to_index에는 존재하지 않는 단어인 'good'이라는 단어가 있습니다.

이와 같이 단어 집합에 존재하지 않는 단어들을 Out-Of-Vocabulary(단어 집합에 없는 단어)의 약자로 'OOV'라 함.   
word_to_index에 'OOV'란 단어를 추가하고 단어 집합에 없는 단어들은 'OOV'의 인덱스로 인코딩

In [10]:
word_to_index['OOV'] = len(word_to_index) + 1

word_to_index를 사용해서 sentences의 모든 단어들을 맵핑되는 정수로 인코딩

In [11]:
encoded = []
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


파이썬의 dictionary 자료형으로 정수 인코딩.  
이보다는 조금 더 쉽게 하기 위해서 Conter, FreqDist, enumerate 또는 keras toknizer를 사용하는 것을 권장.

### Counter 사용하기

In [12]:
from collections import Counter

In [13]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


현재 sentences는 단어 토큰화가 된 결과가 저장되어있음.   
단어 집합(vocabulary)을 만들기 위해 sentences에서 문장 경계인 [,]를 제거하고 단어들을 하나의 리스트로 만들자.

In [14]:
words = sum(sentences, [])
# 위 작업은 words = np.hstack(sentences)로도 수행 가능.
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


python의 Conter()입력으로 사용하면 중복을 제거하고 단어 빈도수 기록

In [15]:
vocab = Counter(words) # 파이썬의 Counter 모듈을 이용하면 단어의 모든 빈도를 쉽게 계산할 수 있습니다.
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


단어를 key로 빈도수가 value로.   
vocab에 단어를 입력하면 빈도수 리턴.

In [16]:
print(vocab["barber"]) # 'barber'라는 단어의 빈도수 출력

8


most_common()은 상위 빈도수를 가진 수의 단어만 리턴.

In [17]:
vocab_size=5
vocab=vocab.most_common(vocab_size) #등장 빈도수가 높은 상위 5개의 단어만 저장
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

높은 빈도수를 가진 단어에 낮은 인덱스 부여

In [18]:
word_to_index = {}
i = 0
for (word, frequency) in vocab :
    i = i+1
    word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### NLTK의 FreqDist 사용하기

NLTK에서는 빈도수 계산도구인 FreqDist()를 지원. 위에서 사용한 Counter()랑 같은 방법으로 사용가능

In [19]:
from nltk import FreqDist
import numpy as np

In [20]:
# np.hstack으로 문장 구분을 제거하여 입력으로 사용 . ex) ['barber', 'person', 'barber', 'good' ... 중략 ...
vocab = FreqDist(np.hstack(sentences))

In [21]:
print(vocab["barber"]) # 'barber'라는 단어의 빈도수 출력

8


vocab에 key=단어, value=빈도수 저장, most_common()은 상위 빈도수를 가진 주어진 수의 단어만 리턴.

In [22]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

빈도수가 높ㅇ느 단어일수록 낮은 정수 인덱스를 부여.    
그런데 이번에는 enumerate()를 사용해 좀 더 짧은 코드로 인덱스 부여

In [23]:
word_to_index = {word[0] : index + 1 for index, word in enumerate(vocab)}
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


이렇듯 인덱스를 부여할 때는 enumerate()를 사용하는 것이 편리. enumerate()에 대해서 간단히 소개

### enumerate 이해하기

enumerate()는 순서가 잇는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아 인덱스를 순차적으로 함께 리턴하는 특징이 있음.

In [24]:
test=['a', 'b', 'c', 'd', 'e']
for index, value in enumerate(test): # 입력의 순서대로 0부터 인덱스를 부여함.
  print("value : {}, index: {}".format(value, index))

value : a, index: 0
value : b, index: 1
value : c, index: 2
value : d, index: 3
value : e, index: 4


리스트의 모든 토큰에 대해 인덱스가 순차적으로 증가되며 부여됨.

## 케라스(Keras)의 텍스트 전처리

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [26]:
sentences=[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

단어 토큰화까디 수행된 텍스트데이터와 동일한 데이터를 사용.

In [27]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences) # fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성한다.

fit_on_text()는 입력한 텍스트로부터 단어 빈도수가 높은 순으로 낮은 정수 인덱스를 부여.   
각 단어에 인덱스가 어떻게 부여되었는지를 보려면 word_index를 사용

In [28]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


각 단어가 카운트를 수행하였을 때 몇 개였는지를 보고자 하면 word_counts를 사용

In [29]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


texts_to_sequences()는 입력으로 들어온 코퍼스에 대하여 각 단어를 이미 정해진 인덱스로 변환

In [30]:
print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


앞서는 빈도수가 높은 단어 n개만을 사용하기 위해 most_common()을 사용. 케라스 토크나이저에서는   
tokenizer=Tokenizer(num_words=숫자)와 같은 방법으로 빈도수가 높은 상위 몇 개 단어만 사용하겠다 지정할 수 있음.

In [31]:
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1) # 상위 5개 단어만 사용
tokenizer.fit_on_texts(sentences)

num_words에서 +1을 더해서 값을 넣어주는 이유는 num_words는 숫자를 0부터 카운트합니다. 만약 5를 넣으면 0 ~ 4번 단어 보존을 의미하게 되므로 뒤의 실습에서 1번 단어부터 4번 단어만 남게됩니다. 그렇기 때문에 1 ~ 5번 단어까지 사용하고 싶다면 num_words에 숫자 5를 넣어주는 것이 아니라 5+1인 값을 넣어주어야 합니다.

실질적으로 숫자 0에 지정된 단어가 존재하지 않는데도 케라스 토크나이저가 숫자 0까지 단어 집합의 크기로 산정하는 이유는 NLP에서 패딩(Paddnig)이라는 작업 때문. 따라서 케라스 토크나이저를 사용할 때는 숫자 0도 단어 집합의 크기로 고려해야한다.

이제 다시 word_index

In [32]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [33]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


상위 5개 단어만 사용하겠다 하는데 13개 단어가 출력됨.   
실제 적용은 text_to_sequences를 사용할 때 적용됨.

In [34]:
print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


만약 word_index와 word_counts에서도 지정된 num_words만큼의 단어만 남기고 싶다면 아래의 코드도 방법.

In [35]:
tokenizer = Tokenizer() # num_words를 여기서는 지정하지 않은 상태
tokenizer.fit_on_texts(sentences)

In [37]:
vocab_size = 5
words_frequency = [w for w,c in tokenizer.word_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거
for w in words_frequency:
    del tokenizer.word_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
    del tokenizer.word_counts[w] # 해당 단어에 대한 카운트 정보를 삭제
print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


케라스 토크나이저는 기본적으로 단어 집합에 없는 단어인 OOV에 대해 단어를 정수로 바꾸는 과정에서 아예 단어를 제거한다는 특징이 있음.단어 집합에 없는 단어들은 OOV로 간주하여 보존하고 싶다면 Tokenizer의 인자 oov_token을 사용.

In [38]:
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
# 빈도수 상위 5개 단어만 사용. 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
tokenizer.fit_on_texts(sentences)

oov를 사용하기로 했다면 케라스토크나이저는 기본적으로 'oov'를 인덱스의 1로 함.

In [39]:
print('단어 OOV의 인덱스 : {}'.format(tokenizer.word_index['OOV']))

단어 OOV의 인덱스 : 1


코퍼스에 대해 정수인코딩

In [40]:
print(tokenizer.texts_to_sequences(sentences))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


빈도수 상위 5개 단어는 2~6까지의 인덱스, 그 외 단어에 없는 'good'과 같은 단어들은 전부 'OOV'의 인덱스인 1로 인코딩됨.